In [8]:
from flask import Flask, render_template, request, redirect, url_for, flash
from ipynb.fs.full.db_config import db_connection
import mysql.connector

In [9]:
app = Flask(__name__, template_folder=r'C:\New folder\Event_Registration\templates')
app.secret_key = "ABRACADABRA" 

In [10]:
@app.route('/create_data',methods=['GET','POST'])
def create_data():
    if request.method == 'POST':
        first_name = request.form['First_name']
        last_name = request.form['Last_name']
        DOB = request.form['dob']
        phone_number = request.form['Phone_number']
        address = request.form['address']
        
        try:
            connection = db_connection()
            cursor = connection.cursor()
            cursor.execute("""
                INSERT INTO Registration (First_name, Last_name,DOB, Phone_number, Address)
                VALUES (%s, %s, %s, %s, %s)
            """,(first_name,last_name, DOB, phone_number, address) )
            connection.commit()
        except mysql.connector.Error as err:
            flask(f"Failed to insert into MySQL table, Error: {err}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
    return render_template('create_label.html')


In [11]:
@app.route('/')
def read_data():
    try:
        connection = db_connection()
        cursor = connection.cursor()
        query = "SELECT * FROM registration"
        cursor.execute(query)
        results = cursor.fetchall()
    except mysql.connector.Error as err:
        flask(f"Error readinf data from MySQl table: {err}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
    return render_template('registrations.html',registrations= results)


In [12]:
@app.route('/update_data/<int:id>',methods=['GET','POST'])
def update_data(id):
    connection = db_connection()
    cursor = connection.cursor()
    query = "UPDATE Registration SET "
    cursor.execute("SELECT * FROM registration WHERE ID = %s", (id,))
    registration = cursor.fetchone()
    updates = []
    params = []
    if request.method == 'POST':
        updates.append("First_name = %s")
        params.append(request.form['First name'])
        updates.append("Last_name = %s")
        params.append(request.form['Last name'])
        updates.append("Phone_number = %s")
        params.append(request.form['phone'])
        updates.append("Address = %s")
        params.append(request.form['address'])
        try:
    
            query += ", ".join(updates) + " WHERE ID = %s"
            params.append(id)
    
            cursor.execute(query, tuple(params))
            connection.commit()
            flash('Registration updated successfully!')
            return redirect(url_for('read_data'))
    
        except mysql.connector.Error as err:
            flask(f"Error to update record to database: {err}")
        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
    return render_template('update.html',registration=registration)


In [13]:
@app.route('/delete_data/<int:id>', methods=['POST'])
def delete_data(id):
    try:
        connection = db_connection()
        cursor = connection.cursor()
        query = "DELETE FROM registration WHERE ID = %s"
        cursor.execute(query, (id,))
        connection.commit()
        flash("Registration deleted successfully!")
    except mysql.connector.Error as err:
        flash(f"Error: {err}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
    return redirect('registrations.html')


In [14]:
if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Oct/2024 01:34:39] "GET / HTTP/1.1" 200 -
